In [1]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
bike_speed = 10
fly_speed = 15
start_distance = 20

In [10]:
class Mover():
    def __init__(self, pos, velocity, t=0):
        self.velocity = velocity
        self.set_reference_position(pos, t)
        
    def set_reference_position(self, pos, t=0):
        self.init_pos = pos - t * self.velocity
        
    def position(self, t):
        return self.init_pos + self.velocity * t
    
    def collision_time(self, other):
        if abs(self.velocity - other.velocity) == 0.0:
            return None
        return (other.init_pos - self.init_pos) / (self.velocity - other.velocity)
    
    def turn_around_at(self, t):
        pos = self.position(t)
        self.velocity = -self.velocity
        self.set_reference_position(pos, t)        

In [24]:
# initialize the bike objects
bike1 = Mover(pos=0, velocity=bike_speed)
bike2 = Mover(pos=start_distance, velocity=-bike_speed)

# initialize the fly object at T=0
t = 0
fly = Mover(pos=bike1.position(t), velocity=fly_speed)

# Keep track of the fly's position at the times when its velocity changes
fly_position = [(t, fly.position(t))]

# ...and the total distance it has flown
fly_distance = 0.0

while len(fly_position) < 8:
    # find the time when the fly next encounters a bike
    t1 = fly.collision_time(bike1)
    t2 = fly.collision_time(bike2)
    if t1 > t2:
        turn_t = t1
    else:
        turn_t = t2
    fly_position.append((turn_t, fly.position(turn_t)))
    fly_distance +=  abs(fly.position(turn_t) - fly.position(t))
    fly.turn_around_at(turn_t)
    
    t = turn_t
    print("T={:6.4f}h  bike1: {:6.3f}mi fly: {:6.3f}mi  bike2: {:6.3f}mi  fly distance= {:6.4f}".format(
        t, bike1.position(t), fly.position(t), bike2.position(t), fly_distance))

T=0.8000h  bike1:  8.000mi fly: 12.000mi  bike2: 12.000mi  fly distance= 12.0000
T=0.9600h  bike1:  9.600mi fly:  9.600mi  bike2: 10.400mi  fly distance= 14.4000
T=0.9920h  bike1:  9.920mi fly: 10.080mi  bike2: 10.080mi  fly distance= 14.8800
T=0.9984h  bike1:  9.984mi fly:  9.984mi  bike2: 10.016mi  fly distance= 14.9760
T=0.9997h  bike1:  9.997mi fly: 10.003mi  bike2: 10.003mi  fly distance= 14.9952
T=0.9999h  bike1:  9.999mi fly:  9.999mi  bike2: 10.001mi  fly distance= 14.9990
T=1.0000h  bike1: 10.000mi fly: 10.000mi  bike2: 10.000mi  fly distance= 14.9998


In [25]:
t = [item[0] for item in fly_position]
pfly = [item[1] for item in fly_position]
pbike1 = [bike1.position(ti) for ti in t]
pbike2 = [bike2.position(ti) for ti in t]

fig = figure(title="The Bikes and the Fly", x_axis_label='time (hours)', y_axis_label='position (miles)')

# add a line renderer with legend and line thickness
fig.line(t, pfly, legend="fly", line_width=2, line_color="red")
fig.line(t, pbike1, legend="bike1", line_width=2, line_dash="2 2", line_color="lightblue")
fig.line(t, pbike2, legend="bike2", line_width=2, line_dash="2 2", line_color="gray")

# show the results
show(fig)